In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-driver

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-18 20:26:03--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-10-18 20:26:03 (11.4 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [ ]:
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [ ]:
# Filter for reviewed products
votes_df = vine_df.filter('total_votes>=20')
votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter for mostly helpful reviews
helpful_reviews_df = votes_df.filter('helpful_votes/total_votes>=.5')
helpful_reviews_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter for vine reviews
vine_reviews_df = helpful_reviews_df.filter(helpful_reviews_df['vine'] == 'Y')
vine_reviews_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
not_vine_reviews_df = helpful_reviews_df.filter(helpful_reviews_df['vine']=='N')
not_vine_reviews_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [23]:
print(f"Total helpful reviews: {helpful_reviews_df.count()}")
helpful_5_df = helpful_reviews_df.filter(helpful_reviews_df['star_rating']==5)
print(f"5-Star helpful reviews: {helpful_5_df.count()}")
print(f"% of total were paid 5-star reviews: {(helpful_5_df.filter(helpful_5_df['vine']=='Y')).count()/helpful_reviews_df.count()*100:.2f}%")
print(f"% of total were unpaid 5-star reviews: {(helpful_5_df.filter(helpful_5_df['vine']=='N')).count()/helpful_reviews_df.count()*100:.2f}%")

Total helpful reviews: 14537
5-Star helpful reviews: 8246
% of total were paid 5-star reviews: 0.23%
% of total were unpaid 5-star reviews: 56.49%


In [24]:
print(f"% of paid reviews that were 5-star: {(helpful_5_df.filter(helpful_5_df['vine']=='Y')).count()/vine_reviews_df.count()*100:.2f}%")

% of paid reviews that were 5-star: 56.67%


In [25]:
print(f"% of unpaid reviews that were 5-star: {(helpful_5_df.filter(helpful_5_df['vine']=='N')).count()/not_vine_reviews_df.count()*100:.2f}%")

% of unpaid reviews that were 5-star: 56.72%
